In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import copy

In [37]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dataloader
def get_data_loaders(train_dir, val_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# model
# baseline
class BaselineEfficientNet(nn.Module):
    def __init__(self, num_classes=6):
        super(BaselineEfficientNet, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False  # freeze
        num_features = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

class Model(nn.Module):
    def __init__(self, num_classes=6):
        super(Model, self).__init__()
        self.feature_extractor = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.classifier = models.efficientnet_b0(pretrained=True)
        num_features = self.classifier.classifier[1].in_features
        self.classifier.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            features = self.feature_extractor(x)
            features = features.view(features.size(0), -1)
            outputs = self.classifier(features)
        return outputs

# train
class Trainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, scheduler, device, patience=5, warmup_steps=0):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.best_model_wts = copy.deepcopy(model.state_dict())
        self.best_accuracy = 0.0
        self.patience = patience
        self.warmup_steps = warmup_steps
        self.early_stop = False
        self.counter = 0
        self.best_score = None

    def train(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0

            for step, (inputs, labels) in enumerate(self.train_loader):
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                # warm-up steps
                if step < self.warmup_steps:
                    lr_scale = min(1.0, float(step + 1) / self.warmup_steps)
                    for pg in self.optimizer.param_groups:
                        pg['lr'] = lr_scale * self.optimizer.defaults['lr']

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()

                running_loss += loss.item()

            train_loss = running_loss / len(self.train_loader)
            val_loss, val_accuracy = self.validate()

            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

            # early stopping
            self._early_stopping(val_loss)
            if self.early_stop:
                print("Early stopping triggered")
                break

            if val_accuracy > self.best_accuracy:
                self.best_accuracy = val_accuracy
                self.best_model_wts = copy.deepcopy(self.model.state_dict())

        self.model.load_state_dict(self.best_model_wts)
        return self.model

    def validate(self):
        self.model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in self.val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        return val_loss / len(self.val_loader), accuracy

    def _early_stopping(self, val_loss):
        score = -val_loss
        if self.best_score is None or score > self.best_score:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True


In [38]:
# setup
train_loader, val_loader = get_data_loaders('C:/uoft/1517/content/content/lung_ct_augmented/train', 'C:/uoft/1517/content/content/lung_ct_augmented/val')
model = Model(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-6)

# train model
trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, scheduler, device, patience=5, warmup_steps=100)
#show the sahpe of train_loader
print(len(train_loader))

#print out first five pictures of train_loader
for i, (inputs, labels) in enumerate(train_loader):
    print(inputs.shape)
    print(labels)
    if i == 5:
        break

    


trained_model = trainer.train(num_epochs=20)

# save model checkpoint
torch.save(trained_model.state_dict(), 'resnet_efficientnet_finetuned.pth')


# load model
trained_model.load_state_dict(torch.load('resnet_efficientnet_finetuned.pth'))
trained_model.eval()



1406
torch.Size([32, 3, 256, 256])
tensor([3, 3, 3, 3, 3, 4, 3, 1, 3, 0, 4, 1, 4, 4, 0, 2, 0, 4, 0, 4, 0, 1, 0, 4,
        4, 1, 4, 2, 0, 3, 0, 0])
torch.Size([32, 3, 256, 256])
tensor([3, 1, 4, 0, 4, 1, 1, 0, 0, 4, 0, 3, 1, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 3,
        3, 1, 0, 1, 2, 0, 4, 1])
torch.Size([32, 3, 256, 256])
tensor([4, 3, 3, 0, 3, 4, 2, 0, 3, 4, 1, 4, 3, 2, 2, 4, 1, 1, 1, 2, 0, 0, 2, 0,
        1, 2, 4, 4, 2, 0, 2, 4])
torch.Size([32, 3, 256, 256])
tensor([0, 3, 1, 0, 2, 0, 1, 3, 0, 1, 0, 0, 1, 4, 0, 0, 0, 1, 4, 1, 1, 0, 3, 3,
        3, 0, 3, 1, 0, 4, 2, 4])
torch.Size([32, 3, 256, 256])
tensor([3, 4, 1, 3, 0, 2, 1, 3, 4, 1, 0, 0, 3, 1, 1, 2, 1, 0, 1, 1, 1, 3, 2, 0,
        3, 2, 3, 2, 1, 3, 1, 3])
torch.Size([32, 3, 256, 256])
tensor([0, 2, 4, 0, 2, 1, 3, 4, 4, 2, 0, 1, 4, 4, 3, 4, 2, 4, 1, 2, 3, 4, 0, 0,
        0, 3, 2, 3, 4, 3, 0, 3])


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 2048]